In [36]:
import pandas as pd
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
import math
from operator import itemgetter
import gc

In [37]:
# Read the CSV file into a DataFrame
path = 'C:\\Users\\ritaf\\Documents\\ESTUDOS\\NOVA IMS\\3º Ano\\Text Mining\\Projeto\\new_data\\'
data = pd.read_csv(path + 'lyrics_fdist_lem_stem_2500.csv')

In [38]:
data = data[['lyrics_string_fdist', 'tag']]

In [39]:
# Create a dictionary to store data for each genre
genre_freqs = {}

# Create a list to store all tokens from all genres
all_tokens = []

# Iterate through unique genres
for genre in data['tag'].unique():
    # Create a subset of the DataFrame for the current genre
    genre_df = data.loc[data['tag'] == genre].drop(columns=['tag'])
    
    # Drop NaN values in 'lyrics_string_fdist'
    genre_df.dropna(subset=['lyrics_string_fdist'], inplace=True)
    
    # Join the 'lyrics_string_fdist' column and tokenize
    genre_lyrics = ' '.join(list(genre_df['lyrics_string_fdist']))
    genre_tokens = word_tokenize(genre_lyrics)
    
    # Append tokens to the list for overall frequency distribution
    all_tokens.extend(genre_tokens)

    # Calculate frequency distribution for the current genre
    genre_freq = FreqDist(genre_tokens)
    
    # Store the data in the dictionary
    genre_freqs[f'freq_{genre}'] = genre_freq

# Calculate overall frequency distribution
overall_freq = FreqDist(all_tokens)

# Add the overall frequency distribution to the dictionary
genre_freqs['freq_all'] = overall_freq

# Access the frequency distributions for each genre and the overall frequency using genre_data
# For example: genre_data['freq_pop'], genre_data['freq_rap'], genre_data['overall_freq'], etc.


In [40]:
# ESTA PODE TER PALAVRAS REPETIDAS


# def log_ratio(genre_freqs, n=20):
#     # Get the overall frequency distribution for all genres
#     all_freq = genre_freqs['freq_all']

#     # Create a dictionary to store the log ratios for each genre
#     genre_log_ratios = {}

#     for genre, genre_freq in genre_freqs.items():
#         # Skip the overall frequency distribution
#         if genre == 'freq_all':
#             continue

#         # Calculate total number of words in genre and in all genres
#         total_genre_words = genre_freq.N()
#         total_all_words = all_freq.N()

#         # Calculate the genre's 100 top words
#         genre_freq_100 = genre_freq.most_common(100)

#         # Calculate log ratios for the 100 top words
#         log_ratios = {
#             word: math.log(((freq + 1) / (total_genre_words + 1)) / ((all_freq[word] + 1) / (total_all_words + 1)))
#             for word, freq in genre_freq_100
#         }

#         # Sort log ratios
#         sorted_log_ratios = sorted(log_ratios.items(), key=itemgetter(1), reverse=True)

#         # Store the top log ratios for the genre
#         genre_log_ratios[genre] = sorted_log_ratios[:n]

#     return genre_log_ratios

# # Assuming genre_freqs is a dictionary with frequency distributions for each genre
# # (including an entry with key 'overall_freq' for the overall frequency distribution)
# # Example usage:
# top_words = log_ratio(genre_freqs)

# # Print top words for each genre
# for genre, words in top_words.items():
#     print(f"Top words for {genre}: {words}")


In [41]:
# ESTA GARANTE QUE AS PALAVRAS SÃO ÚNICAS

def log_ratio(genre_freqs, n=20):
    # Get the overall frequency distribution for all genres
    all_freq = genre_freqs['freq_all']

    # Create a dictionary to store the log ratios for each genre
    genre_log_ratios = {}

    # Create a set to keep track of selected words across genres
    selected_words_set = set()

    for genre, genre_freq in genre_freqs.items():
        # Skip the overall frequency distribution
        if genre == 'freq_all':
            continue

        # Calculate total number of words in genre and in all genres
        total_genre_words = genre_freq.N()
        total_all_words = all_freq.N()

        # Calculate the genre's 100 top words
        genre_freq_100 = genre_freq.most_common(n*5)

        # Calculate log ratios for the 100 top words
        log_ratios = {
            word: math.log(((freq + 1) / (total_genre_words + 1)) / ((all_freq[word] + 1) / (total_all_words + 1)))
            for word, freq in genre_freq_100
        }

        # Sort log ratios
        sorted_log_ratios = sorted(log_ratios.items(), key=itemgetter(1), reverse=True)

        # Select the top words that are not already selected
        selected_words = []
        for word, ratio in sorted_log_ratios:
            if word not in selected_words_set:
                selected_words.append((word, ratio))
                selected_words_set.add(word)

                # Break once n unique words are selected for the genre
                if len(selected_words) == n:
                    break

        # Store the top log ratios for the genre
        genre_log_ratios[genre] = selected_words

    return genre_log_ratios


In [42]:
# Finding the top words
top_words = log_ratio(genre_freqs, n=50)

# Taking a look at the top words for each genre
for genre, words in top_words.items():
    print(f"Top words for {genre}: {words}")

Top words for freq_rock: [('darkness', 0.8936300339633755), ('burn', 0.8548411286382759), ('burning', 0.82825537656492), ('death', 0.7760902149204332), ('blood', 0.7661558987959288), ('lies', 0.7308767238521007), ('fear', 0.7050072864982663), ('rise', 0.6721993939585033), ('broken', 0.6405339112741715), ('alive', 0.6223168050899422), ('dead', 0.6152011021884276), ('inside', 0.6009252919273419), ('prechorus', 0.5809816617226533), ('away', 0.5677597721397807), ('end', 0.5490948272168785), ('sun', 0.5460633062634245), ('ground', 0.5435512554272298), ('fire', 0.5401145592169599), ('bridge', 0.5303989700415128), ('hide', 0.5296885270749826), ('dark', 0.5293135589636127), ('save', 0.5249852621891892), ('earth', 0.5226294858785401), ('war', 0.5157109004425655), ('behind', 0.5138546620160673), ('fight', 0.51057239190304), ('eyes', 0.501319821385536), ('sky', 0.498965961006384), ('everyone', 0.4981615766700522), ('falling', 0.4980977830103464), ('die', 0.49638555698730513), ('alone', 0.47729156

In [43]:
words_only = [word for sublist in top_words.values() for word, _ in sublist]
unique_word_count = len(set(words_only))
print("Number of unique words:", unique_word_count)
print(words_only)

Number of unique words: 300
['darkness', 'burn', 'burning', 'death', 'blood', 'lies', 'fear', 'rise', 'broken', 'alive', 'dead', 'inside', 'prechorus', 'away', 'end', 'sun', 'ground', 'fire', 'bridge', 'hide', 'dark', 'save', 'earth', 'war', 'behind', 'fight', 'eyes', 'sky', 'everyone', 'falling', 'die', 'alone', 'waiting', 'lost', 'light', 'hell', 'fall', 'left', 'comes', 'chorus', 'control', 'far', 'nothing', 'ive', 'stand', 'pain', 'soul', 'words', 'forever', 'youre', 'niggas', 'nigga', 'gang', 'hoes', 'bitch', 'bitches', 'lil', 'rap', 'yall', 'ayy', 'shit', 'dick', 'fuckin', 'aye', 'pussy', 'block', 'wit', 'ima', 'tryna', 'fuck', 'gon', 'hook', 'ass', 'cash', 'imma', 'bag', 'pop', 'aint', 'gettin', 'huh', 'check', 'flow', 'yea', 'yuh', 'hit', 'fucked', 'nah', 'money', 'bout', 'fucking', 'cuz', 'damn', 'smoke', 'pull', 'top', 'mad', 'intro', 'thats', 'game', 'talking', 'na', 'sing', 'tonight', 'dance', 'kiss', 'someone', 'love', 'heart', 'lonely', 'stars', 'sweet', 'feels', 'cry', '

In [49]:
data['lyrics_string_fdist'].value_counts()

tell would like lyrics song make find lyrics first help lyrics song would great help                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [52]:
data_lyr = data['lyrics_string_fdist']

In [54]:
data_lyr.dropna(inplace=True)
data_lyr.value_counts()

tell would like lyrics song make find lyrics first help lyrics song would great help                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
#data_lyr = data.lyrics_string_fdist

def count_vectorizer_to_df(train_lst_strs):

    # Create an instance of the CountVectorizer class - Default vectorizer does not remove stop words
    vectorizer = CountVectorizer(vocabulary=words_only)

    # Fit the vectorizer to the text data and transform the text data into a frequency matrix
    frequency_matrix = vectorizer.fit_transform(train_lst_strs)


    # Convert the frequency matrix to a Pandas DataFrame
    train_df = pd.DataFrame(frequency_matrix.toarray(), columns=vectorizer.get_feature_names_out())

    return train_df

train_df = count_vectorizer_to_df(data_lyr).dropna()
train_df

,darkness,burn,burning,death,blood,lies,fear,rise,broken,alive,...,fact,means,women,least,dear,first,lady,business,went,mother
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,3,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134952,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134953,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134954,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
134955,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
